1: To mount the drive

In [ ]:
%reset -f
from google.colab import drive 
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


2: To install category_encoders - for pre-processing, not really needed as the code is commented out

In [ ]:
!pip install category_encoders

3: The function below is used to get the data from the files using the pandas library

In [ ]:
import pandas as pd
from category_encoders import OrdinalEncoder  # convert categorical features, if non it won't change the data

def get_values_df(filename, CapType, print_data_bool):
    df = pd.read_csv(filename, encoding= 'unicode_escape')
    df.to_csv(filename, header=["Weight", "Thickness", "Porosity%", CapType], index=False)
    df.to_csv(filename, header=["Weight", "Thickness", "Porosity%", CapType], index=False)

    if print_data_bool == 'yes':
        print(df)
    return df

4: Splits df into X and Y 

In [ ]:
def get_values_XY(df, CapType):
    df.dropna(subset = [CapType], inplace=True)
    y_df=df[df.columns[column_of_prediction]].to_frame() # specify output
    X_df=df[df.columns[0:column_of_prediction]] # anything except the output is regressor
    categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']
    # encoder = OrdinalEncoder(cols=categorical_features).fit(X_df)
    # X_df=encoder.transform(X_df)

    return X_df, y_df

5: PCA below, first need to install PCA factor map

In [ ]:
pip install -i https://test.pypi.org/simple/ variable-factor-map-Huy-Bui==0.0.3

Looking in indexes: https://test.pypi.org/simple/


In [ ]:
from sklearn.decomposition import PCA
from variable_factor_map import pca_map 
from sklearn import datasets
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px


def PCA_function(X_df,show_bool, print_bool):
    pca = PCA(n_components=2, svd_solver='full')
    pca.fit(X_df)
    pca_values = pca.components_
    X_df_PCA = pca.transform(X_df)
    exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

    px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
    )

    if print_bool == 'yes':
        print('PCA explained variance ratio:', pca.explained_variance_ratio_)
        print('PCA singular values:', pca.singular_values_)
        print('PCA data shape: ', X_df_PCA.shape)
        print('Initial data shape:', X_df.shape)
        print('PCA components:', pca.components_, '\n')
        

    if show_bool == 'yes':
        pca_map(X_df, figsize=(10,10), sup='WMG Data', print_values= False)
        plt.show()

    return X_df_PCA, pca.explained_variance_ratio_, pca.singular_values_, pca.components_



6: Models and parameter grids below

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ExpSineSquared
from sklearn.linear_model import ARDRegression

#Gradient Boosting
GB = GradientBoostingRegressor()
param_grid_GB = {'n_estimators': [10, 100, 500],
              'learning_rate': [0.1, 0.05, 0.02],
              'max_depth': [4],
              'max_features': ["log2", "sqrt"],
              'criterion': ["friedman_mse", "mae"],
              'min_samples_split': [3],
              'min_samples_leaf': [3],
              'max_features': [1.0]}

#SVR
SVR_model = SVR(kernel = 'rbf')
# param_grid_SVR_model={'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),"C": [1e0, 1e1, 1e2, 1e3, 1e4],
#                 "gamma": np.logspace(-2, 3, 10),'degree' : [3,8]}
param_grid_SVR_model={"C": [1e0, 1e1, 1e2, 1e3, 1e4],
                "gamma": np.logspace(-2, 3, 10),'degree' : [3,8]}


#MLP - NN
MLP = MLPRegressor(alpha = 0.06, solver = "lbfgs", hidden_layer_sizes=3, learning_rate_init=0.1)
param_grid_MLP = {'hidden_layer_sizes': [i for i in range(1,5)],
                  'activation': ["relu"],
                  'solver': ["lbfgs", "sgd", "adam"],
                  'learning_rate': ['constant'],
                  'learning_rate_init': [i for i in [0.01,0.03,0.06,0.1]],
                  'power_t': [0.5],
                  'alpha': [i for i in [0.0001, 0.001,0.01]], #0.00001, 0.0001, 0.001, 0.01
                  #0.0001 was used for C20
                  'max_iter': [1000000],
                  'early_stopping': [False],
                  'warm_start': [False]}

#Gaussian Process Regressor
GPR = GaussianProcessRegressor()            
param_grid_GPR = [{"alpha":  [1e0, 1e-1, 1e-2, 1e-3],"kernel": [RBF(l) for l in np.logspace(-1, 1, 2)]
                   }, {"alpha":  [1e0, 1e-1, 1e-2, 1e-3],"kernel": [DotProduct(sigma_0) for sigma_0 in np.logspace(-1, 1, 2)]},
                   {"alpha":  [1e0, 1e-1, 1e-2, 1e-3]}]
# "kernel": [ExpSineSquared(l, p)+0.001*kernels.Identity()
#                          for l in np.logspace(-2, 2, 10)
#                          for p in np.logspace(0, 2, 10)]

ARD = ARDRegression()
param_grid_ARD = [{'alpha_1': [1e-1, 1e-2, 1e-3, 1e-5]}, {'alpha_2': [1e-1, 1e-2, 1e-3, 1e-5]}]


7. To save estimators, parameters and results

In [ ]:
import csv
def save_estimators_parameters_results(AOrC, CapType, C_Rate, index_filename, model_name_list, index_model, PCA_bool, all_parameters, all_estimators, results):
    with open(
            r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC + '_' + CapType + '/results/I_O_' + AOrC + '_' + CapType + '_parameters_' +
            C_Rate[
                index_filename] + '_' + model_name_list[index_model] + '_' + PCA_bool + '.csv',
            'w') as f:
        for key in all_parameters.keys():
            f.write("%s, %s\n" % (key, all_parameters[key]))
    with open(
            r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC + '_' + CapType + '/results/I_O_' + AOrC + '_' + CapType + '_estimators_' +
            C_Rate[
                index_filename] + '_' + model_name_list[index_model] + '_' + PCA_bool + '.csv',
            'w') as f:
        for key in all_parameters.keys():
            f.write("%s, %s\n" % (key, all_estimators[key]))
    
    np.savetxt(
        r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC + '_' + CapType + '/results/I_O_' + AOrC + '_' + CapType + '_results_' +
        C_Rate[index_filename] + '_' + model_name_list[index_model] + '_' + PCA_bool + '.csv',
        results,
        delimiter=",")  # uncomment these two lines if the results for the individual C_rates are needed in individual files

8. Functions for feature importance for automatic relevance determination

In [ ]:
from matplotlib import pyplot
def get_plots_ARD_coefs(estimator_, n_features):   
      relevant_features = np.arange(0,n_features)
      plt.figure(figsize=(6, 5))
      plt.title("Weights of the model")
      plt.plot(estimator_.coef_, color='darkblue', linestyle='-', linewidth=2,
         label="ARD estimate")
      plt.xlabel("Features")
      plt.ylabel("Values of the weights")
      plt.legend(loc=1)
      plt.show()

      plt.figure(figsize=(6, 5))
      plt.title("Histogram of the weights")
      plt.hist(estimator_.coef_, bins=n_features, color='navy', log=True)
      plt.scatter(estimator_.coef_[relevant_features], np.full(len(relevant_features), 5.),
            color='gold', marker='o', label="Relevant features")
      plt.ylabel("Features")
      plt.xlabel("Values of the weights")
      plt.legend(loc=1)
      plt.show()

      plt.figure(figsize=(6, 5))
      plt.title("Marginal log-likelihood")
      plt.plot(estimator_.scores_, color='navy', linewidth=2)
      plt.ylabel("Score")
      plt.xlabel("Iterations") 
      plt.show()    

def get_FI(regr, X_df, FI_plot_bool):    
    if model == GB: 
      importance = regr.best_estimator_.feature_importances_
      if print_bool == 'yes':
        for i,v in enumerate(importance):
          print('Feature: %0d, Score: %.5f' % (i,v))
      pyplot.bar([x for x in range(len(importance))], importance)
      if FI_plot_bool == 'yes':  
        pyplot.show()
    if model == SVR_model:
      importance = regr.best_estimator_.dual_coef_
    if model == GPR:
      importance = regr.best_estimator_.alpha_
    if model == ARD:
      importance = regr.best_estimator_.coef_ 
      n_features = X_df.shape[1]-1
      if FI_plot_bool == 'yes':
          get_plots_ARD_coefs(regr.best_estimator_, n_features)
    
    if print_bool == 'yes':
        print('Feature importance: \n', importance)

9. Functions used for training  below

In [ ]:
from IPython.display import display_html

# apply the maximum absolute scaling in Pandas using the .abs() and .max() methods
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column] / df_scaled[column].abs().max()
    return df_scaled

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def train(X_df, y_df, model, param_grid_in_use, print_bool, FI_bool, FI_plot_bool):
    
    _GS = GridSearchCV(model, param_grid=param_grid_in_use, n_jobs = -1)

    X, y = make_regression(n_samples=200, random_state=0)
    X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=0)
    regr = _GS.fit(X_train, y_train.values.ravel())
    # regr = model.fit(X_train, y_train.values.ravel()) # use this when not using gridsearch

    # regr.predict(New_data) #for prediction on data outside dataset
    
    # y_pred = pd.DataFrame(regr.predict(X_test), columns=['pred'], index=X_test.index) # use this when not using gridsearch
    y_pred = pd.DataFrame(regr.predict(X_test), columns=['pred'])


    # comment out to make reading easier
    # display_side_by_side(y_pred, y_test)
    error = y_pred.values-y_test.values
    if print_bool == 'yes':
      print('regr score is', regr.score(X_test, y_test))
      print('RMSE', np.mean(np.absolute(error)))

    if FI_bool == 'yes':
      get_FI(regr, X_df, FI_plot_bool)

    return regr.score(X_test, y_test), np.mean(np.absolute(error)), regr.best_score_, regr.best_estimator_, regr.best_params_
    # return regr.score(X_test, y_test), np.mean(np.absolute(error)) #use this when not using grid search


10. The function below is to plot all the results

In [ ]:
from matplotlib.patches import Patch

def get_plots(AOrC, CapType, result_name, C_Rate, model_name_list, PCA_bool, analytics, stds, save_plot_bool):
    color_list = ['red', 'green', 'blue', 'black']
    x_pos = np.arange(len(result_name))
    # Build the plot
    fig, axs = plt.subplots(len(C_Rate))
    # fig.suptitle('All C Rates Results', fontsize = 20)
    fig.subplots_adjust(top=0.8, hspace=0.5)
    for i in range(len(C_Rate)):
        for j in range(len(model_name_list)):
            axs[i].bar(x_pos*len(model_name_list)+j, analytics[i,j], yerr=stds[i,j], align='center', alpha=1/len(model_name_list), ecolor='black', color =color_list[j] , capsize=10, label = C_Rate[i])
        axs[i].set_ylabel('Results', fontsize = 5)
        axs[i].set_xticks(x_pos, result_name)
        axs[i].set_xlabel(', '.join(map(str, result_name)), fontsize=10)
        axs[i].set_title('Results of models on I_O_' + AOrC + '_' + CapType + '_plots_' + PCA_bool +'_'+ C_Rate[i], fontsize = 15)
        axs[i].yaxis.grid(True)

    plt.subplots_adjust(left=0.1,
                    bottom=1.0, 
                    right=0.9, 
                    top=7.0, 
                    wspace=0.4, 
                    hspace=0.4)

    # map names to colors
    cmap = dict(zip(model_name_list, color_list))
    # create the rectangles for the legend
    patches = [Patch(color=v, label=k) for k, v in cmap.items()]
    # add the legend
    plt.legend(title='Type of Model', labels=model_name_list, handles=patches, bbox_to_anchor=(1.04, 0.5), loc='center left', borderaxespad=0, fontsize=15, frameon=False)

    # Save the figure and show
    plt.tight_layout(pad=100.0) 
    plt.show()
    if save_plot_bool == 'yes':
        plt.savefig(r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC + '_' + CapType + '/results/I_O_' + AOrC + '_' + CapType + '_plots_' + '_' + model_name_list[index_model] + '_'+ PCA_bool +'_'.join(result_name) + '_'+ PCA_bool + '.png', dpi=600)


11: Initialise

In [ ]:
# filename = r'/gdrive/My Drive/FUSE data/Experimental-Dataset.csv'
C_Rate = ["C20", "C5", "C2", "1C", "2C", "5C", "10C"]
CapType_list = ["Cap", "VCap", "GCap"]
PCA_bool_list = ["non-PCA", "PCA"]
AOrC_list = ["Anode", "Cathode"]
AOrC = AOrC_list[0]
CapType = CapType_list[0]
column_of_prediction = 3
# filename = r'/gdrive/My Drive/FUSE data/I_O_'+AOrC+'_'+CapType+'/I_O_'+AOrC+'_'+CapType+'_'+C_Rate[index_filename]+'.csv'

#options for model are GB, SVR_model, GPR, MLP
model_list = [ARD, GB, SVR_model, GPR]
model_name_list = ['ARD', 'GB', 'SVR_model', 'GPR']
param_grid_list = [param_grid_ARD, param_grid_GB, param_grid_SVR_model, param_grid_GPR]

12: Add header to file names - this has to be run twice due to df.to_csv function

In [ ]:
for index_AOrC in range(len(AOrC_list)):
    for index_CapType in range(len(CapType_list)):
            for index_filename in range(len(C_Rate)):
                print_data_bool = 'no' #print data bool is set to no 
                filename = r'/gdrive/My Drive/FUSE data/I_O_'+AOrC_list[index_AOrC]+'_'+CapType_list[index_CapType]+'/I_O_'+AOrC_list[index_AOrC]+'_'+CapType_list[index_CapType]+'_'+C_Rate[index_filename]+'.csv'
                # print(filename)
                #get values
                df = get_values_df(filename, CapType_list[index_CapType], print_data_bool)

In [ ]:
for index_AOrC in range(len(AOrC_list)):
    for index_CapType in range(len(CapType_list)):
            for index_filename in range(len(C_Rate)):
                print_data_bool = 'no' #print data bool is set to no 
                filename = r'/gdrive/My Drive/FUSE data/I_O_'+AOrC_list[index_AOrC]+'_'+CapType_list[index_CapType]+'/I_O_'+AOrC_list[index_AOrC]+'_'+CapType_list[index_CapType]+'_'+C_Rate[index_filename]+'.csv'
                # print(filename)
                #get values
                df = get_values_df(filename, CapType_list[index_CapType], print_data_bool)

13: Main run is below

In [ ]:
import time
import csv
import statistics as st
import numpy as np
for index_AOrC in range(len(AOrC_list)):
    for index_PCA_bool in range(len(PCA_bool_list)):
        for index_CapType in range(len(CapType_list)):
            compiled_results = []
            result_name = ['R^2', 'RMSE', 'Time']
            analytics = np.zeros((len(C_Rate), len(model_name_list), len(result_name)))
            stds = np.zeros((len(C_Rate), len(model_name_list), len(result_name)))
            for index_filename in range(len(C_Rate)):
                for index_model in range(len(model_name_list)):

                    # option to print everything out
                    print_bool = 'no'
                    # option to print data
                    print_data_bool = 'no'

                    # option to use PCA or non-PCA data
                    PCA_bool = PCA_bool_list[index_PCA_bool]
                    # option to show the PCA map
                    show_PCA_map_bool = 'no'
                    # option to scale features
                    scale_bool = 'no'
                    
                    # option to save the final results
                    save_estimators_parameters_results_bool = 'no'
                    # option to save the final results all compiled into one file
                    save_compiled_results_bool = 'yes'
                    # option to save the final results plot - this option has issues, need to manually save plots
                    save_plot_bool = 'no'
                    
                    # option to print feature importance plot
                    FI_bool = 'no'
                    # option to show FI plots
                    FI_plot_bool = 'no'

                    # initialise lists and choose model
                    scores = []
                    RMSEs = []
                    all_parameters = {}
                    all_estimators = {}
                    times = []
                    model = model_list[index_model]
                    model_name = model_name_list[index_model]
                    param_grid_in_use = param_grid_list[index_model]

                    #printing filename to check progress
                    print('\nI_O_' + AOrC_list[index_AOrC] + '_' + CapType_list[index_CapType] + '_' + C_Rate[index_filename] + '_' + model_name_list[
                        index_model] + '_' + PCA_bool)
                    filename = r'/gdrive/My Drive/FUSE data/I_O_' + AOrC_list[index_AOrC] + '_' + CapType_list[index_CapType] + '/I_O_' + AOrC_list[index_AOrC] + '_' + CapType_list[index_CapType] + '_' + \
                               C_Rate[index_filename] + '.csv'
                    # print(filename)

                    # get values
                    df = get_values_df(filename, CapType_list[index_CapType], print_data_bool)
                    X_df, y_df = get_values_XY(df, CapType_list[index_CapType])

                    # get only numerical values and call the maximum_absolute_scaling function, currently not used
                    if scale_bool == 'yes':
                        X_df = maximum_absolute_scaling(X_df)

                    # run PCA
                    X_df_PCA, PCA_explained_variance_ratio_, PCA_singular_values_, PCA_components_ = PCA_function(X_df, show_PCA_map_bool, print_bool)

                    # to print parameters and group them in an array to print the learning curve later
                    epochs = 3
                    for i in range(epochs):

                        print('Run:', i + 1)
                        #To assign PCA or non-PCA X to train
                        if PCA_bool == 'PCA':
                            X_df_dummy = X_df_PCA
                        else:
                            X_df_dummy = X_df

                        #Start training    
                        start = time.time()
                        score, error, best_score, best_estimator, best_params = train(X_df_dummy, y_df, model, param_grid_in_use, print_bool, FI_bool, FI_plot_bool)
                        # score, error = train(X_df_PCA, y_df) #for use when not using grid search
                        stop = time.time()
                        print(f"Training time: {stop - start}s")
                        times.append(stop - start)

                        # storing results from multiple runs
                        scores.append(score) #R^2 scores
                        RMSEs.append(error) #RMSE scores
                        all_estimators[str(i)] = best_estimator  # comment out when not using grid search
                        all_parameters[str(i)] = best_params  # comment out when not using grid search

                    # storing mean and stdev of results
                    compiled_results.append([st.mean(scores), st.stdev(scores), st.mean(RMSEs), st.stdev(RMSEs), st.mean(times),st.stdev(times)]) #for the final table
                    results = np.asarray([scores, RMSEs, times])  # individual file results
                    analytics[index_filename, index_model] = [st.mean(scores), st.mean(RMSEs), st.mean(times)] #to store all mean values in array
                    stds[index_filename, index_model] = [st.stdev(scores), st.stdev(RMSEs), st.stdev(times)] #to store all std values in array
                    if print_bool == 'yes':
                        print(results)

                    #to write the data R^2, RMSE and best estimators, parameters and results parameters to .csv files, only change save_estimators&parameters&results_bool to 'yes' if there is space to store everything
                    if save_estimators_parameters_results_bool == 'yes':
                        save_estimators_parameters_results(AOrC_list[index_AOrC], CapType_list[index_CapType], C_Rate, index_filename, model_name_list, index_model, PCA_bool, all_parameters, all_estimators, results)
                        print('Saved estimators, parameters and results for '+model_name_list[index_model]+' '+PCA_bool+' run on '+filename)

            # the time plot is separated from the other result metrics
            get_plots(AOrC_list[index_AOrC], CapType_list[index_CapType], result_name[0:-1], C_Rate, model_name_list, PCA_bool, analytics[:, :, 0:-1], stds[:, :, 0:-1],
                      save_plot_bool)
            get_plots(AOrC_list[index_AOrC], CapType_list[index_CapType], [result_name[-1]], C_Rate, model_name_list, PCA_bool, analytics[:, :, -1], stds[:, :, -1],
                      save_plot_bool)

            #save compiled results
            compiled_results_array = np.asarray(compiled_results)
            if save_compiled_results_bool == 'yes':
                np.savetxt(
                    r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC_list[index_AOrC] + '_' + CapType_list[index_CapType] + '/results/I_O_' + AOrC_list[index_AOrC] + '_' + CapType_list[index_CapType] + '_Compiled_Results_' + PCA_bool + '.csv', compiled_results_array, delimiter=",")


I_O_Anode_Cap_C20_ARD_non-PCA
Run: 1
Training time: 1.2765576839447021s
Run: 2
Training time: 0.27225184440612793s
Run: 3
Training time: 0.2829318046569824s

I_O_Anode_Cap_C20_GB_non-PCA
Run: 1


KeyboardInterrupt: ignored

This final section allows you to get the plots again. It make it easier to change the plot settings and not needing to run the training section again. 

In [ ]:
save_plot_bool = 'no'
get_plots(result_name[0:-1], C_Rate, model_name_list, PCA_bool, analytics[:,:,0:-1], stds[:,:,0:-1], save_plot_bool)
get_plots([result_name[-1]], C_Rate, model_name_list, PCA_bool, analytics[:,:,-1], stds[:,:,-1], save_plot_bool)

save_compiled_results_bool = 'no'
if save_compiled_results_bool == 'yes':
    np.savetxt(r'/gdrive/MyDrive/FUSE data/I_O_' + AOrC + '_' + CapType + '/results/I_O_'+ AOrC + '_' + CapType + '_Compiled_Results_' + PCA_bool + '.csv', compiled_results_array, delimiter=",")
